<a href="https://colab.research.google.com/github/xuzhuzzz/Repo2/blob/master/tuduis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
# 准备数据集
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=torchvision.transforms.ToTensor())

# 检查数据集大小
train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练集大小：{}".format(train_data_size))
print("测试集大小：{}".format(test_data_size))

100%|██████████| 170M/170M [00:13<00:00, 12.4MB/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified
训练集大小：50000
测试集大小：10000


In [ ]:
import torch
from torch import nn


class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = nn.Sequential(
            # 输入图片大小为 32x32x3,通道数为3，卷积核大小为5，padding为2，输出大小为32x32x32
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
        )
    def forward(self, x):
        x = self.model1(x)
        return x

In [ ]:
# 使用 DataLoader 加载数据集
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# 定义设备是否使用GPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tudui.to(device)


# 加载数据集
train_loader = DataLoader(train_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

# 定义模型
tudui = Tudui()
# 使用GPU
tudui = tudui.cuda()

# 定义损失函数, 这里使用交叉熵损失函数
loss_func = nn.CrossEntropyLoss()
loss_func = loss_func.cuda() # 放在GPU上运行
# 定义学习率,方便修改
learning_rate = 1e-2 # 1e-2=1*10^-2
# 定义优化器
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)


# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
num_epochs = 10

# 添加tensorboard日志记录
writer = SummaryWriter(log_dir='../logs/tudui_gpu')

# 开始训练
for epoch in range(num_epochs):
    print("-------------第{}轮训练开始了----------".format(epoch+1))

    # 训练模型
    for data in train_loader:
        # 准备数据
        imgs, targets = data
        # 转到GPU
        imgs = imgs.cuda()
        targets = targets.cuda()

        # 前向传播
        outputs = tudui(imgs)
        # 计算损失
        loss = loss_func(outputs, targets)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        # 更新参数
        optimizer.step()

        # 记录训练次数
        total_train_step += 1
        # 打印训练信息，每100次打印一次
        if total_train_step % 100 == 0:
            print("训练次数:{}, 损失:{}".format(total_train_step, loss.item()))
            # 记录训练loss到tensorboard
            writer.add_scalar('train_loss', loss.item(), total_train_step)

    # 怎么知道模型训练是否达到预期效果呢?
    # 这里我们让模型在测试集上测试一下，以测试集上的损失或者准确率来评估模型的效果
    # 这里我们只测试一次，实际应用中可以多次测试，取平均值

    # 测试步骤开始
    tudui.eval() # 测试模式，只对某些层有作用，如dropout层
    # 可以设置一个变量，用于记录总的测试损失或者准确率
    total_test_loss = 0
    total_test_acc = 0 # 整体正确的
    with torch.no_grad(): # 关闭梯度计算，节省内存，不会更新参数
        for data in test_loader:
            # 准备数据
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            # 前向传播
            outputs = tudui(imgs)
            # 计算损失
            loss = loss_func(outputs, targets)
            # 有时候只计算损失，不能评估模型的效果，这时可以计算准确率
            total_test_loss += loss.item()
            # 计算准确率（分类问题，特有的指标）
            total_test_acc += (outputs.argmax(dim=1) == targets).sum().item()


    print("第{}轮训练结束, 整体测试集上的loss:{}".format(epoch+1, total_test_loss))
    print("第{}轮训练结束, 整体测试集上的准确率:{}".format(epoch+1, total_test_acc/len(test_data)))
    # 记录测试次数
    total_test_step += 1
    writer.add_scalar('test_loss', total_test_loss, total_test_step)
    writer.add_scalar('test_accuracy', total_test_acc/len(test_data), total_test_step)

    # 保存每一轮的模型参数
    # torch.save(tudui, "tudui_{}.pth".format(epoch+1))

# 训练结束，关闭tensorboard
writer.close()


-------------第1轮训练开始了----------
训练次数:100, 损失:2.2948508262634277
训练次数:200, 损失:2.2845137119293213
训练次数:300, 损失:2.268329381942749
训练次数:400, 损失:2.188488721847534
训练次数:500, 损失:2.116820812225342
训练次数:600, 损失:2.063305139541626
训练次数:700, 损失:1.9892157316207886
第1轮训练结束, 整体测试集上的loss:309.5932240486145
第1轮训练结束, 整体测试集上的准确率:0.3007
-------------第2轮训练开始了----------
训练次数:800, 损失:1.8531948328018188
训练次数:900, 损失:1.8174786567687988
训练次数:1000, 损失:1.927711009979248
训练次数:1100, 损失:1.9707622528076172
训练次数:1200, 损失:1.6489378213882446
训练次数:1300, 损失:1.5936248302459717
训练次数:1400, 损失:1.7229770421981812
训练次数:1500, 损失:1.7304039001464844
第2轮训练结束, 整体测试集上的loss:288.7738070487976
第2轮训练结束, 整体测试集上的准确率:0.3404
-------------第3轮训练开始了----------
训练次数:1600, 损失:1.6987910270690918
训练次数:1700, 损失:1.6287415027618408
训练次数:1800, 损失:1.87433922290802
训练次数:1900, 损失:1.699959635734558
训练次数:2000, 损失:1.9124797582626343
训练次数:2100, 损失:1.484344720840454
训练次数:2200, 损失:1.4470312595367432
训练次数:2300, 损失:1.7537457942962646
第3轮训练结束, 整体测试集上的loss:263.723644